In [7]:
# https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-your-first-pipeline
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

In [8]:
ws.get_details()

{'applicationInsights': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourcegroups/myresourcegroup/providers/microsoft.insights/components/llachmanmlwork7726509491',
 'containerRegistry': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourceGroups/MyResourceGroup/providers/Microsoft.ContainerRegistry/registries/llachmanmlwo5de1ae17',
 'creationTime': '2019-08-10T02:16:58.7668266+00:00',
 'description': '',
 'friendlyName': '',
 'id': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourceGroups/MyResourceGroup/providers/Microsoft.MachineLearningServices/workspaces/llachmanMLWorkspace',
 'identityPrincipalId': '2166891c-3629-4f17-8f8e-72c1b4d8d9fb',
 'identityTenantId': 'c7970e8d-6d07-4b20-bf16-eba4864e56ef',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourcegroups/myresourcegroup/providers/microsoft.keyvault/vaults/llachmanmlwork6145098727',
 'location': 'centralus',
 'name': 'llachmanMLWorkspace',
 'stor

In [9]:
print(ws.compute_targets)

{'llachmanvm1': DsvmCompute(workspace=Workspace.create(name='llachmanmlworkspace', subscription_id='165934ca-a3be-4df3-842b-332997fcecd9', resource_group='myresourcegroup'), name=llachmanvm1, id=/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourceGroups/myresourcegroup/providers/Microsoft.MachineLearningServices/workspaces/llachmanmlworkspace/computes/llachmanvm1, type=VirtualMachine, provisioning_state=Succeeded, location=centralus, tags=None), 'cpucluster': AmlCompute(workspace=Workspace.create(name='llachmanmlworkspace', subscription_id='165934ca-a3be-4df3-842b-332997fcecd9', resource_group='myresourcegroup'), name=cpucluster, id=/subscriptions/165934ca-a3be-4df3-842b-332997fcecd9/resourceGroups/myresourcegroup/providers/Microsoft.MachineLearningServices/workspaces/llachmanmlworkspace/computes/cpucluster, type=AmlCompute, provisioning_state=Succeeded, location=centralus, tags=None)}


In [10]:
# Default datastore - stores the data for the pipeline to access
def_data_store = ws.get_default_datastore()

# Get the blob storage associated with the workspace
def_blob_store = Datastore(ws, "workspaceblobstore")

# Get file storage associated with the workspace
def_file_store = Datastore(ws, "workspacefilestore")

In [11]:
def_blob_store.upload_files(
    ["./data/20news.pkl"],
    target_path="20newsgroups",
    overwrite=True)

Uploading an estimated of 1 files
Uploading ./data/20news.pkl
Uploaded ./data/20news.pkl, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_e97f72386896406eb043b019334ffeee

In [23]:
from azureml.data.data_reference import DataReference

# A data source in a pipeline is represented by a DataReference object, 
# which points to data that lives in or is accessible from a datastore
blob_input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="test_data",
    path_on_datastore="20newsgroups/20news.pkl")

In [24]:
# Intermediate data is represented by a PipelineData object, and introduces a data dependency between steps
output_data1 = PipelineData(
    "output_data1",
    datastore=def_blob_store,
    output_name="output_data1")

In [25]:
# Ensure a compute_target is attached to our ML Workspace
compute_name = "cpucluster"
vm_size = "STANDARD_D2_V2"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
    else:
        print('Could not find compute target', compute_target, type(compute_target))

Found compute target: cpucluster


In [27]:
project_folder = 'project'

# PythonScriptStep is a built-in pipeline step, which runs a Python script in a specified compute target
trainStep = PythonScriptStep(
    script_name="train.py",
    arguments=["--input", blob_input_data, "--output", output_data1],
    inputs=[blob_input_data],
    outputs=[output_data1],
    compute_target=compute_target,
    source_directory=project_folder
)

In [28]:
# list of steps to run
compareModels = [trainStep, extractStep, compareStep]

# Build the pipeline
pipeline1 = Pipeline(workspace=ws, steps=[compareModels])

NameError: name 'extractStep' is not defined

In [ ]:
dbStep = NotebooVMStep(
    name=compute_name,
    inputs=[step_1_input],
    outputs=[step_1_output],
    num_workers=1,
    notebook_path=notebook_path,
    notebook_params={'myparam': 'testparam'},
    run_name='demo run name',
    compute_target=databricks_compute,
    allow_reuse=False
)
# List of steps to run
steps = [dbStep]

# Build the pipeline
pipeline1 = Pipeline(workspace=ws, steps=steps)

In [ ]:
# Submit the pipeline to be run
pipeline_run1 = Experiment(ws, 'Compare_Models_Exp').submit(pipeline1)
pipeline_run1.wait_for_completion()

In [ ]:
# Parameterize the pipeline before publishing so you can run it with different inputs later
pipeline_param = PipelineParameter(
  name="pipeline_arg",
  default_value=10)

In [ ]:
# Add this PipelineParameter object as a parameter to any of the steps in the pipeline as follows:
compareStep = PythonScriptStep(
  script_name="compare.py",
  arguments=["--comp_data1", comp_data1, "--comp_data2", comp_data2, "--output_data", out_data3, "--param1", pipeline_param],
  inputs=[ comp_data1, comp_data2],
  outputs=[out_data3],
  target=compute_target,
  source_directory=project_folder)

In [ ]:
# Publish this pipeline that will accept a parameter when invoked
published_pipeline1 = pipeline_run1.publish_pipeline(
     name="My_Published_Pipeline",
     description="My Published Pipeline Description",
     version="1.0")